# Dependency locality and information locality

This exercise aims to reproduce and evaluate the methodology related to assess dependency locality and information locality in natural languages.

We expect you to work in binoms and return a document commenting your answers. This document will be supported by the code in the notebook.

The goal is to figure out how to carry out experiments like those described in the  paper *Modeling word and morpheme order in natural language as an efficient tradeoff of memory and surprisal* by Hahn, Degen and Futrell.

We will investigate how to design language models with limited memory and how to use them to answer questions on natural and manipulated datasets. The first part of the exercise is methodological. It amounts to design and set up non standard language models. The second part of the exercise aims to investigate how to use these models to answer scientific questions.  




**The answers to the exercises have to be returned as a written document (approx 5 pages). The code will be given in appendix. The grading uses mainly the text.**

## Memory limited neural language models

We consider two types of language models, first Neural Network Language models (NNLM) then Recurrent Neural Network Language Models (RNNLM)
These are models predicting the $i-$th word given the $k$ previous ones. We provide an example implementation for an NNLM and you will have to design an implementation for the RNNLM in pytorch

### The tokenizer

The first step is to implement a tokenizer. The tokenizer plays two roles
 - it splits the input string into tokens
 - it maps the tokens to integer codes

We design a class that does exactly that. Note that we have to add some artificial tokens to handle unknown words for instance

The example tokenizer follows loosely the [HuggingFace tokenizer](https://huggingface.co/docs/transformers/en/main_classes/tokenizer) interface

In [1]:
import torch
import numpy as np
import seaborn as sns
import os

In [2]:
from models.MarkovianNN import NNLM
from utils.tokenizer import NaiveTokenizer
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import numpy as np

In [3]:
# Toyish example
# Create the tokenizer with some known vocabulary

# observe how the unknown vocabulary is encoded
# observe how padding is encoded too

tokenizer = NaiveTokenizer("Language models are cool .".split())
codes     = tokenizer("Language models are not so cool .")
toks      = tokenizer.tokenize("Language models are not so cool .")

print("codes",codes)
print("toks",toks)

#Example of padding
print("\npadded codes")
batch = [tokenizer(sentence) for sentence in ["Language models are cool .","Language models are not so cool ."]]
print(tokenizer.pad_batch(batch))


codes [2, 3, 4, 0, 0, 5, 6]
toks ['Language', 'models', 'are', 'not', 'so', 'cool', '.']

padded codes
tensor([[2, 3, 4, 5, 6, 1, 1],
        [2, 3, 4, 0, 0, 5, 6]])


### Dataset and Dataloader

[Dataset and Dataloader](https://pytorch.org/docs/stable/data.html) are pytorch classes that are used to load efficiently a dataset by batches on one or several GPUs. Here we provide a naive introductory example for language modeling


In [4]:
from torch.utils.data import Dataset,DataLoader
from utils.dataLoader import NgramsLanguageModelDataSet
from utils.sentenceProcessing import normalize


#Dummy example to see what it does

zebra_dataset = """
There are five houses.
The Englishman lives in the red house.
The Spaniard owns the dog.
Coffee is drunk in the green house.
The Ukrainian drinks tea.
The green house is immediately to the right of the ivory house.
The Old Gold smoker owns snails.
Kools are smoked in the yellow house.
Milk is drunk in the middle house.
The Norwegian lives in the first house.
The man who smokes Chesterfields lives in the house next to the man with the fox.
Kools are smoked in the house next to the house where the horse is kept.
The Lucky Strike smoker drinks orange juice.
The Japanese smokes Parliaments.
The Norwegian lives next to the blue house.
"""
sentences = [normalize(sent) for sent in zebra_dataset.split('\n') if sent and not sent.isspace()]
tokenizer = NaiveTokenizer(normalize(zebra_dataset).split())
dataset   = NgramsLanguageModelDataSet(5,sentences,tokenizer) #pentagram dataset

print('DataLoader output\n')
dataloader = DataLoader(dataset, batch_size=10, shuffle=True,collate_fn=tokenizer.pad_batch)
for batch in dataloader:
  print(batch)


DataLoader output

tensor([[34, 10, 11, 35, 13],
        [11, 26, 27, 11, 28],
        [ 7, 14, 15, 11, 16],
        [17, 18, 19, 10, 11],
        [ 9, 10, 11, 13, 44],
        [20, 13, 18, 24, 25],
        [ 7, 21, 22, 23,  6],
        [51, 31, 22, 52, 53],
        [10, 11, 39, 13,  6],
        [34, 10, 11, 13, 44]])
tensor([[18, 24, 25, 11, 26],
        [30, 31, 15, 32,  6],
        [43,  9, 10, 11, 13],
        [33,  3, 34, 10, 11],
        [19, 10, 11, 20, 13],
        [38,  9, 10, 11, 39],
        [13, 44, 25, 11, 13],
        [44, 25, 11, 40, 45],
        [11, 48, 18, 49,  6],
        [14, 15, 11, 16,  6]])
tensor([[ 9, 44, 25, 11, 56],
        [40, 45, 11, 46,  6],
        [10, 11, 37, 13,  6],
        [13, 44, 25, 11, 40],
        [ 9, 10, 11, 12, 13],
        [ 8,  9, 10, 11, 12],
        [44, 25, 11, 13, 47],
        [24, 25, 11, 26, 27],
        [ 7, 20, 13, 18, 24],
        [25, 11, 26, 27, 11]])
tensor([[36, 18, 19, 10, 11],
        [10, 11, 35, 13,  6],
        [ 7, 29, 3

**Exercise 1 (1pt)** Read the code above and try to understand it.  Why do we see only the last few tokens of the test sentence ? This is a common issue in many language modeling implementations. Provide a correction to the code and explain what was the cause.



*   This is because we tokenize the sentences into words and store $N$ words and predict the next words. Given the language models are the next-token predcitor, for sentence with a fixed length $T$, only the last $T-N+1$ words will be able to be predicted by the language models and the first $N$ words will never be predicted as it is required for next-token prediction for language models.
*   Below is the correction to code we've made by padding $N$ ids as \<pad\> to the left of the sentence. Note that we factorized our codes into modules for better reusability and readability for later experiments. Our modification is made to the class `NgramsLanguageModelDataSet(Dataset)` within `dataloader.py`.




In [5]:
import torch.nn as nn
from torch.nn.functional import tanh,log_softmax
from models.MarkovianNN import NNLM

In [6]:
lang_model = NNLM(128,tokenizer.vocab_size,128,5-1,tokenizer.pad_id)
lang_model.train(dataloader,40,device='cpu') #or 'cuda' to run on a GPU or 'mps' for latest macos

#predicting with the model

test_set    = NgramsLanguageModelDataSet(5,["The Lucky Strike smoker drinks orange juice ."],tokenizer, left_padding=True)
test_loader = DataLoader(test_set, batch_size=len(test_set), shuffle=False,collate_fn=tokenizer.pad_batch)

for batch in test_loader:
    print(batch)
    predictions = lang_model(batch)
    src    = tokenizer.decode_ngram(batch)
    print(src)
    logits = lang_model(batch)
    print(list(zip(src.split(),logits.tolist())))

tensor([[ 1,  1,  1,  1,  7],
        [ 1,  1,  1,  7, 50],
        [ 1,  1,  7, 50, 51],
        [ 1,  7, 50, 51, 31],
        [ 7, 50, 51, 31, 22],
        [50, 51, 31, 22, 52],
        [51, 31, 22, 52, 53],
        [31, 22, 52, 53,  6]])
The Lucky Strike smoker drinks orange juice .
[('The', -4.212986946105957), ('Lucky', -3.9778690338134766), ('Strike', -4.181881427764893), ('smoker', -7.417435169219971), ('drinks', -0.002360298065468669), ('orange', -0.0015788720920681953), ('juice', -0.0013959197094663978), ('.', -0.0005983707960695028)]


**Exercise 2 (1pt)** Try to understand how the language model  handles unknown words. Is it the same method as GPT-2 ? explain the differences.
Can we easily compare the suprisals returned by the two models ? what is your opinion?

In [7]:
newTest = """**Exercise 3 (3pts)** Given the code above, implement a Recurrent Neural Network Language Model trainable on ngrams. The dataset and the dataloader can be reused as is. The main difference is located in the Module subclass. This time we do not concatenate the embeddings of the past tokens but rather the model uses an RNN module. Pytorch library provides an LSTM module and there are [several tutorials explaining how to implement those models online](https://docs.pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html). Explain what are the main challenges related to implementing LSTM like models in pytorch. Do you manage to run your models on cuda devices ?" \

In order to test to which extent human language minimize dependency length and enforce information locality,
we will compare word order in natural corpus with word order in corpora where word order is somewhat randomized.
The corpora used here are taken from Universal Dependencies.
"""

In [ ]:
# training a scratch-built GPT-2 model on zebra dataset
from transformers import GPT2Config, GPT2LMHeadModel, GPT2TokenizerFast
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

# ----- 1. Build training texts -----
# assume zebra_dataset is an iterable of strings (one sequence each)
train_texts = [normalize(t) for t in zebra_dataset]

# ----- 2. Train GPT-2-style tokenizer on your data -----
base_gpt2_tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

gpt2_tokenizer = base_gpt2_tokenizer.train_new_from_iterator(
    train_texts,
    vocab_size=tokenizer.vocab_size,   # or pick an explicit number like 8000
)

gpt2_tokenizer.add_special_tokens({
    "pad_token": "<|pad|>",
    "eos_token": "<|eos|>",
    "bos_token": "<|bos|>",
})

# ----- 3. Config consistent with THIS tokenizer -----
config = GPT2Config(
    vocab_size=len(gpt2_tokenizer),
    n_layer=4,
    n_head=4,
    n_embd=256,
    bos_token_id=gpt2_tokenizer.bos_token_id,
    eos_token_id=gpt2_tokenizer.eos_token_id,
    pad_token_id=gpt2_tokenizer.pad_token_id,
)

# ----- 4. Randomly initialized GPT-2 LM -----
gpt2_model = GPT2LMHeadModel(config)
gpt2_model.to(device)
gpt2_model.train()

# ----- 5. Dataset using the same tokenizer -----
class LMDataset(Dataset):
    def __init__(self, texts, tokenizer, block_size=128):
        self.texts = texts
        self.tokenizer = tokenizer
        self.block_size = block_size

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        enc = self.tokenizer(
            self.texts[idx],
            return_tensors="pt",
            truncation=True,
            max_length=self.block_size,
            padding="max_length",   # pad to fixed length for batching
        )
        ids = enc["input_ids"].squeeze(0)  # (block_size,)
        return {
            "input_ids": ids,
            "labels": ids.clone(),         # standard causal LM objective
        }

train_ds = LMDataset(train_texts, tokenizer=gpt2_tokenizer)
trainloader = DataLoader(train_ds, batch_size=5, shuffle=True)

optimizer = torch.optim.AdamW(gpt2_model.parameters(), lr=5e-4)

# ----- 6. Training loop -----
num_epochs = 40

for epoch in range(num_epochs):
    pbar = tqdm(trainloader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)
    for batch in pbar:
        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = gpt2_model(**batch)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # show current loss on the bar
        pbar.set_postfix(loss=float(loss.item()))

gpt2_model.eval()

/Users/yingcai/opt/anaconda3/envs/nlp/lib/python3.12/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/Users/yingcai/opt/anaconda3/envs/nlp/lib/python3.12/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <367D4265-B20F-34BD-94EB-4F3EE47C385B> /Users/yingcai/opt/anaconda3/envs/nlp/lib/python3.12/site-packages/torchvision/image.so
  Reason: tried: '/Users/yingcai/opt/anaconda3/envs/nlp/lib/python3.12/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/yingcai/opt/anaconda3/envs/nlp/lib/python3.12/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/yingcai/opt/anaconda3/envs/nlp/lib/python3.12/lib-dynload/../../libjpeg.9.dylib' (no such file), '/Users/yingcai/opt/anaconda3/envs/nlp/bin/../lib/libjpeg.9.dylib' (no such file)'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Epoch 1/40:   0%|          | 0/128 [00:00<?, ?it/s]

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


In [ ]:
import torch.nn.functional as F
# compute the surprisals of each word in the new test set for our markovian model
test_set    = NgramsLanguageModelDataSet(5, [ newTest ], tokenizer, left_padding=True)
test_loader = DataLoader(test_set, batch_size=len(test_set), shuffle=False,collate_fn=tokenizer.pad_batch)
surprisals_all = []
for batch in test_loader:
    logits = lang_model(batch)  # shape (batch_size, vocab_size)
    scr = tokenizer.decode_ngram(batch)
    unk_idx = []
    for i, w in enumerate(scr.split()):
        if w == '<unk>':
            unk_idx.append(i)
    log_probs = logits[unk_idx]  # shape (batch_size, vocab_size)
    surprisals_all.extend(-log_probs.detach().numpy())  # assuming batch_size=1
sns.histplot(surprisals_all, bins=30, label="Unknown words in Hidden Markov Model")
# compute the surprisals of each word in the new test set for gpt-2
inputs = gpt2_tokenizer(newTest, return_tensors="pt")
# Move input tensors to the same device as the model
inputs = {k: v.to(device) for k, v in inputs.items()}
suprisals_all_gpt2 = []
with torch.no_grad():
    outputs = gpt2_model(**inputs, labels=inputs["input_ids"])
    logits = outputs.logits  # shape (batch_size, seq_length, vocab_size)
    targets = inputs["input_ids"][:, 1:]  # shape (batch_size, seq_length)
    log_probs = torch.log_softmax(logits, dim=-1)  # shape (batch_size, seq_length, vocab_size)
    log_probs = log_probs[:, :-1, :].gather(2, targets.unsqueeze(-1)).squeeze(-1)  # shape (batch_size, seq_length)
    suprisals_all_gpt2.extend(-log_probs.detach().cpu().numpy().flatten())
sns.histplot(suprisals_all_gpt2, bins=30, label="GPT-2")
sns.despine()
plt.ylabel("Frequency")
plt.xlabel("Surprisal")
plt.legend()
plt.show()

- Our models treat unkown words as \<unk\>. For every unknown words in the testing set as shown above (blue), it has a lower bound of around 4.
- The GPT-2 use a Byte-Pair Encoding (BPE) tokenizer, which will decompose unknown words into fragments of known tokens, meaning that BPE guarantees any string is tokenizable. In addition, common patterns (e.g., the, ing) become short token sequences but for rare/unseen patterns will stay as small tokens, suggesting that an unkown sequences required longer sequences and therefore giving higher surprisals. For example, if "rareword" is a common pattern in the training corpora, that means this patten can be represented as a single token, hence the surprisal will be $p(rareword|context) = a$. However, if it is less likely, lets say it will be decomposed into $[t_1, t_2, t_3, t_4]$, even the model learns that these tokens are more likely (i.e., $p(t_i) = b > a$), the suprisal will still be larger due to the multiplicative nature of the tokens -- $p(rareword|context)=\prod^n_{i=1}p(t_i) = \sum^n_{i=1} \log p(t_i)$, as shown in orange above.
- The properties of BPE make it very difficult to compare two different approaches. To compare them, first an untrained GPT-2 model has to be trained on the same copera and maintain the same vocab. To capture the unknown sequences that has not been shown to GPT-2 tokenizer, we have to align and concatenate the tokens so that it matches the unknown words in the context, and sum their surprisals for comparison. Then a statistical test is required to test if the KL-divergence between two distributions of the surprisals does not arise from chance, which tell us whether the two distributions are different from each other (perhaps a permutation test)
- To conclude, it is not easy to compare the surprisals between the two distributions as it required sophisticated alignments and design of the GPT-2 and training procedures. From a theoretical point of view, for unknown sequences, most unknown words will recieve a larger surprisal in the case of BPE tokenizer, while the model with a simple <unk> exhibits a more narrower range of surprisal due to weaker dependency of the context.

**Exercise 3 (3pts)** Given the code above, implement a Recurrent Neural Network Language Model trainable on ngrams. The dataset and the dataloader can be reused as is. The main difference is located in the Module subclass. This time we do not concatenate the embeddings of the past tokens but rather the model uses an RNN module. Pytorch library provides an LSTM module and there are [several tutorials explaining how to implement those models online](https://docs.pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html).  

Explain what are the main challenges related to implementing LSTM like models in pytorch. Do you manage to run your models on cuda devices ?

In [ ]:
import torch
from utils.dataLoader import NgramsLanguageModelDataSet
from torch.utils.data import DataLoader
from models.RNNLM import RNNLanguageModel
from models.MarkovianNN import NNLM
from utils.tokenizer import NaiveTokenizer
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import numpy as np
from utils.sentenceProcessing import normalize
from utils.conllu_parser import *
from utils.permutationTest import *
from utils.dependencies import *
from torch.nn.functional import tanh,log_softmax

In [ ]:
# --- 0. Setup Device ---
# Check if CUDA (GPU) is available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- 1. Prepare Data ---
zebra_dataset = """
There are five houses.
The Englishman lives in the red house.
The Spaniard owns the dog.
Coffee is drunk in the green house.
The Ukrainian drinks tea.
The green house is immediately to the right of the ivory house.
The Old Gold smoker owns snails.
Kools are smoked in the yellow house.
Milk is drunk in the middle house.
The Norwegian lives in the first house.
The man who smokes Chesterfields lives in the house next to the man with the fox.
Kools are smoked in the house next to the house where the horse is kept.
The Lucky Strike smoker drinks orange juice.
The Japanese smokes Parliaments.
The Norwegian lives next to the blue house.
"""
sentences = [normalize(sent) for sent in zebra_dataset.split('\n') if sent and not sent.isspace()]
tokenizer = NaiveTokenizer(normalize(zebra_dataset).split())
dataset   = NgramsLanguageModelDataSet(N=5,data=sentences,tokenizer=tokenizer) #pentagram dataset
dataloader = DataLoader(dataset, batch_size=10, shuffle=True,collate_fn=tokenizer.pad_batch)
# Collect all unique words to build tokenizer vocabulary
for batch in dataloader:
    print("Batch from DataLoader:")
    print(batch)

print(f"Dataset size: {len(dataset)} n-grams")
print(f"Number of batches: {len(dataloader)}")
print(f"Vocabulary size: {tokenizer.vocab_size}")


In [ ]:
sentences = [normalize(sent) for sent in zebra_dataset.split('\n') if sent and not sent.isspace()]
print(sentences)
print(normalize(zebra_dataset).split())

In [ ]:
# --- 2. Initialize Model ---
vocab_size = tokenizer.vocab_size
embedding_dim = 64 # Size of word embeddings
hidden_dim = 128   # Size of LSTM hidden states
num_layers = 2     # Number of LSTM layers
dropout_prob = 0.3 # Dropout for regularization
print(f"vocab_size: {vocab_size}")
model = RNNLanguageModel(vocab_size, embedding_dim, hidden_dim, num_layers, dropout_prob)
model.to(device)

In [ ]:
# --- 3. Train the Model ---
num_epochs = 20
learning_rate = 0.005

# Call the train_model method
model.train_model(dataloader, num_epochs, learning_rate, device=device)

In [ ]:
# --- 4. (Optional) Evaluate or Generate Text after Training ---
print("\n--- Model evaluation/generation ---")
model.eval() # Set model to evaluation mode (disables dropout, etc.)

test_set    = NgramsLanguageModelDataSet(5,["The Lucky Strike smoker drinks orange juice ."], tokenizer, left_padding=True)
test_loader = DataLoader(test_set, batch_size=len(test_set), shuffle=False,collate_fn=tokenizer.pad_batch)

print(f"Test set size: {len(test_set)} n-grams")
print(f"batches in test loader: {len(test_loader)}")
for batch in test_loader:
    print(batch)
    predictions, _ = model(batch)
    src    = tokenizer.decode_ngram(batch)
    print(predictions.shape)
    predicted_tokens = torch.argmax(predictions, dim=-1).tolist()
    log_probs = log_softmax(predictions, dim=-1)
    surprisal = -log_probs.gather(1, torch.tensor(predicted_tokens).unsqueeze(-1)).squeeze(-1)
    predicted_words = tokenizer.decode(predicted_tokens)
    print(f'scr: {src}')
    print(f'predicted: {predicted_words}')
    print(f'surprisal: {surprisal.tolist()}')
    

**Main Challenges with LSTMs in PyTorch:**

- For continuous sequences (not just independent n-grams), LSTMs maintain internal hidden and cell states. So we need to pass these states from the end of one batch as the initial states for the next. This requires careful handling, especially with variable sequence lengths and batching. For independent n-grams, as in our current model, this is simplified because each n-gram is a fresh sequence.
- We need to use padding if sequences within a batch have different lengths. To prevent the LSTM from processing padding tokens, `nn.utils.rnn.pack_padded_sequence` and `pad_packed_sequence` are used, which adds complexity to the data pipeline and model's `forward` pass.
- Because of there very reccurent structure, LSTMs can sometimes suffer from exploding or vanishing gradients, especially in deep networks or with long sequences, leading to `NaN` or `inf` loss values. Techniques like gradient clipping are often necessary.
- These condiseration are more real-world issues, but we've been able to run our model on CUDA as a proof of concept.

## Generating artificial datasets

In order to test to which extent human language minimize dependency length and enforce information locality,
we will compare word order in natural corpus with word order in corpora where word order is somewhat randomized.
The corpora used here are taken from Universal Dependencies.


**Exercise 4 (1pt)** Download the UD corpora from the website and choose 10 languages with which you will work. Some of them should be free word order some
of them should be fixed word order. Ideally corpora should be large enough to estimate language models. To figure out whether a language is free or fixed word order you may look at resources like the [World Atlas of Language Structures](https://wals.info)
Write an explanation on how you chose your corpora.

Regarding selectino criteria, we should :

1. ensure the languages are present in UD
2. prioritize languages whose corpora are likely large enough to estimate language models
3. aim for diversity of families/geographic regions to make the results more generalizable.

So we can go for the following UD available languages :

1. **English** – typologically fairly fixed SVO order.
2. **Spanish** – another SVO, fixed‐order example, Romance family.
3. **German** – but typologically more flexible (WALS shows it lacks a single dominant SVO vs SOV) → good “semi‐free” testing case.
4. **Turkish** – typologically strongly SOV/OV (head‐final) and relatively more fixed order (though topic‐marking allows variation) → fixed order side.
5. **Hungarian** – typologically discourse‐configurational with flexible word order → good free‐order candidate.
6. **Russian** – Slavic, moderately flexible word order (case marking allows alternations) → another free‐order candidate.
7. **Japanese** – strongly SOV, fairly fixed in canonical word‐order → fixed side.
8. **Finnish** – Uralic language, fairly flexible word order due to rich morphology → free‐order side.
9. **Arabic (Modern Standard Arabic)** – typologically VSO or SVO depending on dialect, variation makes it interesting → flexible side.
10. **Chinese (Mandarin)** – typologically SVO and relatively fixed in major clauses → fixed side.

### Dowaload UD corpora

In [ ]:
import re
import requests
import time

english = "https://raw.githubusercontent.com/UniversalDependencies/UD_English-CHILDES/master/en_childes-ud-test.conllu"
spanish = "https://raw.githubusercontent.com/UniversalDependencies/UD_Spanish-AnCora/master/es_ancora-ud-test.conllu"
german = "https://raw.githubusercontent.com/UniversalDependencies/UD_German-GSD/master/de_gsd-ud-test.conllu"
turkish = "https://raw.githubusercontent.com/UniversalDependencies/UD_Turkish-GB/master/tr_gb-ud-test.conllu"
hungarian = "https://raw.githubusercontent.com/UniversalDependencies/UD_Hungarian-Szeged/master/hu_szeged-ud-test.conllu"
russian = "https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-GSD/master/ru_gsd-ud-test.conllu"
japanese = "https://raw.githubusercontent.com/UniversalDependencies/UD_Japanese-GSD/master/ja_gsd-ud-test.conllu"
finnish = "https://raw.githubusercontent.com/UniversalDependencies/UD_Finnish-FTB/master/fi_ftb-ud-test.conllu"
arabic = "https://raw.githubusercontent.com/UniversalDependencies/UD_Arabic-PUD/master/ar_pud-ud-test.conllu"
chinese = "https://raw.githubusercontent.com/UniversalDependencies/UD_Chinese-Beginner/master/zh_beginner-ud-test.conllu"

corpora_url = [english, spanish, german, turkish, hungarian, russian,
               japanese, finnish, arabic, chinese]

corpora = []
for i, lang_url in enumerate(corpora_url):
    try:
        response = requests.get(lang_url, timeout=10)
        current = response.text
        corpora.append(current)
        print(f"Corpora sucessfully downloaded {i+1}")
        time.sleep(0.5)
    except requests.exceptions.RequestException as e:
        print(f"Downloading failed at {lang_url}: {e}")
        corpora.append(None)

**Exercise 5 (4pt)** Generate artificial corpora from the natural corpora. Those artificial corpora will have randomized word order.
The `dependency.py` module given in the git may help you read the corpora and shuffle the dependency trees ?
As controlling randomness is a key issue you may wish to consult the paper *Modeling word and morpheme order in natural language as an efficient tradeoff of memory and surprisal*
and decide if the default randomization procedures suit your needs.

- We studied the provided dependencies.py module and learned its sophisticated approach to preserving dependency structure during randomization. The module offers two methods: shuffle_tree() which preserves dependency relationships while randomizing word positions, and shuffle_projective() which maintains projectivity constraints. However, for our information locality research, we developed a custom conllu_parser.py module to have more direct control over word forms and seamless integration.
- We implemented two complementary randomization strategies to test distinct linguistic hypotheses.
    - The first is complete randomization, where word order is entirely shuffled without preserving any structural information. This serves as the strongest baseline to test Hypothesis H1 (Information Locality): natural language word order is optimized to cluster predictively relevant information locally. If $\text{Surprisal}{\text{natural}} < \text{Surprisal}{\text{random}}$, this demonstrates that word order organization reduces prediction uncertainty, as proposed by Hahn et al. (2021).
    - The second strategy is dependency-preserving randomization, inspired by dependencies.py's shuffle_tree() method. This tests Hypothesis H2 (Dependency Length Minimization): the optimization in natural language specifically operates by placing syntactically related words closer together (Futrell et al., 2015).


## Generate artificial corpora

In [ ]:
# Import corpus generation functions from conllu_parser module
# there are the functions to generate 2 types of shuffled corpus
from utils.conllu_parser import (
    generate_natural_corpus,
    generate_complete_random_corpus,
    generate_dep_preserved_corpus
)


In [ ]:
# here we use a part of english corpora as the example of how to generate artifical corpora
en_conllu = corpora[0]
max_sents = 1000
natural = generate_natural_corpus(en_conllu, max_sentences=max_sents)

complete_random = generate_complete_random_corpus(en_conllu, max_sentences=max_sents, seed=42)

dep_preserved = generate_dep_preserved_corpus(en_conllu, max_sentences=max_sents, seed=42)
print(f"Natural: {natural}\nComplete random: {complete_random}\ndependency reserved: {dep_preserved}")

## Dependency Length experiments

**Exercise 6 (5pts)** Try to answer the following questions. Is there a significative difference between dependency length observed in natural corpora with the dependency length observed in corpora with a randomized word order ? Illustrate the differences with relevant plots. Can you identify an hypothesis test that would allow to test the observed differences ? Do languages said to be free word order have indeed longer average dependency lengths ?  How important is the choice of the randomization procedure to reach your conclusion ? You may combine observations from plots and also relevant hypothesis testing to support your answers.

- There is a clear and consistent difference. As shown in our results, all languages exhibit **longer** average dependency lengths after word order randomization in 2 ways. Ex: English: 7.18 (natural) → 21.79 (shuffled) — 203% increase. The evidence that this increased dependency length exists **across all** 10 languages and **different ways of randomization**(Note when we preserved dependency relation, the increased dependency length is **smaller** than the full randominization breaking all relation) strongly supports the hypothesis proposed in the paper: natural language word order is optimized to minimize dependency length, keeping syntactically related words closer together.

- In our results, free word order languages **do not** seem to have longer dependency lengths. Ex: Turkish: 6.27 vs English: 7.18. Russian: 60.29 vs German: 60.15. The data shows that free word order languages do not systematically have longer dependency lengths. In fact, Turkish (free order) has one of the shortest dependency lengths, while Spanish (fixed SVO) has the longest.

- The choice of randomization is crucial for drawing valid conclusions. Without comparing different randomization strategies, we cannot determine which linguistic properties are responsible for the optimization. The complete randomization we used provides strong evidence that natural word order specifically minimizes dependency length.

- In summary: Natural language universally shows **optimized word order** for dependency length minimization, **regardless** of whether the language is classified as "free" or "fixed" word order. The randomization procedure is essential for isolating and identifying this specific optimization principle.

- For statistical testing, we implemented a permutation test to compare the means of dependency lengths between natural corpora and randomized corpora by shuffling the labels 1000 times. And we make a null hypothesis: the average dependency length in randomized dataset is not different from the natural corpora. After running the permutation test, the null hypothesis has a p-value of 0 across 1000 times and thus we can confirm that the null hypothesis is false, then we accept the alternative hypothesis.

- Alternative hypothesis: the average dependency length in randomized dataset is significantly different from the natural corpora.

In [ ]:
import matplotlib.pyplot as plt
from utils.conllu_parser import ConlluParser, DependencyAnalyzer

parser = ConlluParser()
analyzer = DependencyAnalyzer()

# Language mapping
int2lang = {0:"en", 1:"spanish", 2:"des", 3:"turk", 4:"hungarian",
            5:"russia", 6:"jap", 7:"finnish", 8:"arabic", 9:"cn"}

# Store results
results_complete_random = {"natural": {}, "complete_random": {}}

print("=" * 80)
print("Complete Randomization Analysis")
print("=" * 80)

for idx, corpus_text in enumerate(corpora):
    lang_name = int2lang[idx]

    # Parse all sentences (no limit)
    sentences = parser.parse(corpus_text)

    # Dependency length for natural corpus
    natural_totals, natural_avg = analyzer.total_and_average_length(sentences)

    # Complete randomization
    complete_random_sentences = [s.shuffle(seed=42) for s in sentences]
    complete_random_totals, complete_random_avg = analyzer.total_and_average_length(complete_random_sentences)

    # Save results
    results_complete_random["natural"][lang_name] = {
        "avg": natural_avg,
        "totals": natural_totals,
        "n_sentences": len(sentences)
    }
    results_complete_random["complete_random"][lang_name] = {
        "avg": complete_random_avg,
        "totals": complete_random_totals,
        "n_sentences": len(complete_random_sentences)
    }

    # Calculate increase percentage
    increase_pct = ((complete_random_avg / natural_avg) - 1) * 100

    # Print results
    print(f"Language: {lang_name:10} "
          f"Natural: {natural_avg:7.2f} | Complete Random: {complete_random_avg:7.2f} | "
          f"Increase: {increase_pct:6.1f}%")

print("\n" + "=" * 80)

# Plot comparison
langs = list(results_complete_random["natural"].keys())
natural_avgs = [results_complete_random["natural"][l]["avg"] for l in langs]
random_avgs = [results_complete_random["complete_random"][l]["avg"] for l in langs]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Left plot: Bar chart comparison
x = range(len(langs))
ax1.bar([i - 0.2 for i in x], natural_avgs, width=0.4, label="Natural", alpha=0.7, color='steelblue')
ax1.bar([i + 0.2 for i in x], random_avgs, width=0.4, label="Complete Random", alpha=0.7, color='coral')
ax1.set_xlabel("Language", fontsize=12)
ax1.set_ylabel("Average Dependency Length", fontsize=12)
ax1.set_title("Complete Randomization: Natural vs Random Word Order", fontsize=14, fontweight='bold')
ax1.set_xticks(x)
ax1.set_xticklabels(langs, rotation=45, ha='right')
ax1.legend(fontsize=11)
ax1.grid(axis='y', alpha=0.3)

# Right plot: Increase percentage
increases = [((random_avgs[i] / natural_avgs[i]) - 1) * 100 for i in range(len(langs))]
colors = ['crimson' if inc > 300 else 'orange' if inc > 200 else 'gold' for inc in increases]
ax2.barh(langs, increases, color=colors, alpha=0.7)
ax2.set_xlabel("Dependency Length Increase (%)", fontsize=12)
ax2.set_ylabel("Language", fontsize=12)
ax2.set_title("Relative Increase in Dependency Length", fontsize=14, fontweight='bold')
ax2.axvline(x=200, color='red', linestyle='--', alpha=0.5, label='200% threshold')
ax2.legend(fontsize=10)
ax2.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

# Statistical summary
print("\n" + "=" * 80)
print("Statistical Summary (Complete Randomization):")
print("=" * 80)
print(f"Average increase: {sum(increases) / len(increases):.1f}%")
print(f"Minimum increase: {min(increases):.1f}% ({langs[increases.index(min(increases))]})")
print(f"Maximum increase: {max(increases):.1f}% ({langs[increases.index(max(increases))]})")

- p value: the probability that the null hypothesis is true
- null hypothesis: the average dependency length in randomized dataset is not different from the natural corpora.
- threshold: 0.05 -- if p < 0.05, we reject the null hypothesis, we accept the alternative hypothesis
- alternative hypothesis: the average dependency length in randomized dataset is significantly different from the natural corpora.


- Statistical Test:
  - t-test: compare the mean between two groups, assumptions: data from both group follows normal distribution
  - Signed Rank ...
  - permutation test

abs(observed p) < permuted p

In [ ]:
import matplotlib.pyplot as plt
from utils.conllu_parser import ConlluParser, DependencyAnalyzer

parser = ConlluParser()
analyzer = DependencyAnalyzer()

# Language mapping
int2lang = {0:"en", 1:"spanish", 2:"des", 3:"turk", 4:"hungarian",
            5:"russia", 6:"jap", 7:"finnish", 8:"arabic", 9:"cn"}

# Store results
results_dep_preserved = {"natural": {}, "dep_preserved": {}}

print("=" * 80)
print("Dependency-Preserving Randomization Analysis")
print("=" * 80)

for idx, corpus_text in enumerate(corpora):
    lang_name = int2lang[idx]

    # Parse all sentences (no limit)
    sentences = parser.parse(corpus_text)

    # Dependency length for natural corpus
    natural_totals, natural_avg = analyzer.total_and_average_length(sentences)

    # Dependency-preserving randomization
    dep_preserved_sentences = [s.shuffle_preserve_deps(seed=42) for s in sentences]
    dep_preserved_totals, dep_preserved_avg = analyzer.total_and_average_length(dep_preserved_sentences)

    # Save results
    results_dep_preserved["natural"][lang_name] = {
        "avg": natural_avg,
        "totals": natural_totals,
        "n_sentences": len(sentences)
    }
    results_dep_preserved["dep_preserved"][lang_name] = {
        "avg": dep_preserved_avg,
        "totals": dep_preserved_totals,
        "n_sentences": len(dep_preserved_sentences)
    }

    # Calculate increase percentage
    increase_pct = ((dep_preserved_avg / natural_avg) - 1) * 100

    # Print results
    print(f"Language: {lang_name:10} "
          f"Natural: {natural_avg:7.2f} | Dep-Preserved: {dep_preserved_avg:7.2f} | "
          f"Increase: {increase_pct:6.1f}%")

print("\n" + "=" * 80)

# Plot comparison
langs = list(results_dep_preserved["natural"].keys())
natural_avgs = [results_dep_preserved["natural"][l]["avg"] for l in langs]
dep_avgs = [results_dep_preserved["dep_preserved"][l]["avg"] for l in langs]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Left plot: Bar chart comparison
x = range(len(langs))
ax1.bar([i - 0.2 for i in x], natural_avgs, width=0.4, label="Natural", alpha=0.7, color='steelblue')
ax1.bar([i + 0.2 for i in x], dep_avgs, width=0.4, label="Dep-Preserved Random", alpha=0.7, color='mediumpurple')
ax1.set_xlabel("Language", fontsize=12)
ax1.set_ylabel("Average Dependency Length", fontsize=12)
ax1.set_title("Dependency-Preserving Randomization: Natural vs Random Positions", fontsize=14, fontweight='bold')
ax1.set_xticks(x)
ax1.set_xticklabels(langs, rotation=45, ha='right')
ax1.legend(fontsize=11)
ax1.grid(axis='y', alpha=0.3)

# Right plot: Increase percentage
increases = [((dep_avgs[i] / natural_avgs[i]) - 1) * 100 for i in range(len(langs))]
colors = ['darkviolet' if inc > 150 else 'mediumpurple' if inc > 100 else 'plum' for inc in increases]
ax2.barh(langs, increases, color=colors, alpha=0.7)
ax2.set_xlabel("Dependency Length Increase (%)", fontsize=12)
ax2.set_ylabel("Language", fontsize=12)
ax2.set_title("Relative Increase in Dependency Length", fontsize=14, fontweight='bold')
ax2.axvline(x=100, color='purple', linestyle='--', alpha=0.5, label='100% threshold')
ax2.legend(fontsize=10)
ax2.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

# Statistical summary
print("\n" + "=" * 80)
print("Statistical Summary (Dependency-Preserving Randomization):")
print("=" * 80)
print(f"Average increase: {sum(increases) / len(increases):.1f}%")
print(f"Minimum increase: {min(increases):.1f}% ({langs[increases.index(min(increases))]})")
print(f"Maximum increase: {max(increases):.1f}% ({langs[increases.index(max(increases))]})")


In [ ]:
# compare Dep-preserved with complete random
# Plot comparison
langs = list(results_dep_preserved["natural"].keys())
natural_avgs = [results_dep_preserved["natural"][l]["avg"] for l in langs]
dep_avgs = [results_dep_preserved["dep_preserved"][l]["avg"] for l in langs]
random_avgs = [results_complete_random["complete_random"][l]["avg"] for l in langs]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Left plot: Bar chart comparison
x = range(len(langs))
ax1.bar([i - 0.2 for i in x], random_avgs, width=0.4, label="Complete Random", alpha=0.7, color='coral')
ax1.bar([i + 0.2 for i in x], dep_avgs, width=0.4, label="Dep-Preserved Random", alpha=0.7, color='mediumpurple')
ax1.set_xlabel("Language", fontsize=12)
ax1.set_ylabel("Average Dependency Length", fontsize=12)
ax1.set_title("Dependency-Preserving Randomization: Complete Random vs Dep-Preserved Random Positions", fontsize=14, fontweight='bold')
ax1.set_xticks(x)
ax1.set_xticklabels(langs, rotation=45, ha='right')
ax1.legend(fontsize=11)
ax1.grid(axis='y', alpha=0.3)

# Right plot: Increase percentage
increases = [((dep_avgs[i] / random_avgs[i]) - 1) * 100 for i in range(len(langs))]
colors = ['darkviolet' if inc > 150 else 'mediumpurple' if inc > 100 else 'plum' for inc in increases]
ax2.barh(langs, increases, color=colors, alpha=0.7)
ax2.set_xlabel("Dependency Length Increase (%)", fontsize=12)
ax2.set_ylabel("Language", fontsize=12)
ax2.set_title("Relative Increase in Dependency Length", fontsize=14, fontweight='bold')
ax2.axvline(x=100, color='purple', linestyle='--', alpha=0.5, label='100% threshold')
ax2.legend(fontsize=10)
ax2.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

# Statistical summary
print("\n" + "=" * 80)
print("Statistical Summary (Dependency-Preserving Randomization relative to Complete Random):")
print("=" * 80)
print(f"Average increase: {sum(increases) / len(increases):.1f}%")
print(f"Minimum increase: {min(increases):.1f}% ({langs[increases.index(min(increases))]})")
print(f"Maximum increase: {max(increases):.1f}% ({langs[increases.index(max(increases))]})")

permutation_test_results = {}
num_permutations = 1000
# randomization test between dep-preserved and complete random
print("\n" + "=" * 80)
print("Permutation Test Results (Dep-Preserved vs Complete Random):")
print("=" * 80)
from utils.permutationTest import permutation_test
for idx, lang in enumerate(langs):
    dep_preserved_totals = results_dep_preserved["dep_preserved"][lang]["totals"]
    complete_random_totals = results_complete_random["complete_random"][lang]["totals"]
    p_value, _, _= permutation_test(dep_preserved_totals, complete_random_totals, num_permutations=num_permutations)
    print(f"Language: {lang:10} | p-value: {p_value:.4f}")
    

## Information Locality experiments

**Exercise 7(4 pts)** Mirroring the previous exercise, we aim to assess for locality effects in natural language without measuring dependency lengths but rather using **information theoretic measures** computed from language models. To test the relevance of the information locality theorem, train language models with different memory sizes (NNLM, RNNLM or others) and for each of your corpora, compare the average surprisals of the models as a function of the memory size. What do you observe ? how do you interpret the results ? How would you compare the results across languages with different word order properties? is it straightforward ? explain the problematics and illustrate your conclusions with plots and/or hypothesis tests.

$$
S(t) = H(X \mid Y) = -\sum_{x,y} p(x,y)\,\log p(x \mid y) \\
I(t) = S(t-1) -S(t)
$$


need to compute the joint probability matric and conditional distribution

```
dep_preserved_sentences = [s.shuffle_preserve_deps(seed=42) for s in sentences]
dep_preserved_totals, dep_preserved_avg = analyzer.total_and_average_length(dep_preserved_sentences)
complete_random_sentences = [s.shuffle(seed=42) for s in sentences]
complete_random_totals, complete_random_avg = analyzer.total_and_average_length(complete_random_sentences)
```

In [ ]:
from torch.utils.data import random_split
import os
os.getcwd()
if not os.path.exists("./data"):
    os.mkdir("./data")
print(os.path.exists("./data"))

In [ ]:
# Defined dataset for training language models
def prepare_sentences_for_datasets(sentences):
    return [normalize(sent) for sent in sentences if sent and not sent.isspace()]
def prepare_setence_for_tokenizer(sentences):
    return normalize(' '.join(sentences)).split()
def obtain_sentences_from_conllu(sentences):
    return [s.to_text() for s in sentences]
def prepare_datasets(corpora_text, tokenizer, memory_size):
    parser = ConlluParser()
    analyzer = DependencyAnalyzer()
    sentences = parser.parse(corpora_text)
    complete_random_sentences = [s.shuffle(seed=42) for s in sentences]
    dep_preserved_sentences = [s.shuffle_preserve_deps(seed=42) for s in sentences]

    sentences = obtain_sentences_from_conllu(sentences)
    complete_random = obtain_sentences_from_conllu(complete_random_sentences)
    dep_preserved = obtain_sentences_from_conllu(dep_preserved_sentences)

    naturalSentences = NgramsLanguageModelDataSet(memory_size, prepare_sentences_for_datasets(sentences), tokenizer)
    completeRandomSentences = NgramsLanguageModelDataSet(memory_size, prepare_sentences_for_datasets(complete_random), tokenizer)
    depPreservedSentences = NgramsLanguageModelDataSet(memory_size, prepare_sentences_for_datasets(dep_preserved), tokenizer)

    return naturalSentences, completeRandomSentences, depPreservedSentences

def split_dataset(dataset, split_ratio=0.8):
    total_size = len(dataset)
    train_size = int(total_size * split_ratio)
    val_size = total_size - train_size
    return random_split(dataset, [train_size, val_size])

In [ ]:
def train_and_evaluate(model, train_loader, val_loader, num_epochs, learning_rate, device, type_desc):
    model.to(device)
    if type_desc == "RNNLM":
        model.train_model(train_loader, num_epochs, learning_rate, device=device)
    elif type_desc == "NNLM":
        model.train(train_loader, epochs=num_epochs, device=device)
    
    # evaluation
    surprisals = []
    if type_desc == "RNNLM":
        surprisal = model.evaluate_rnn(model, val_loader, device)

    elif type_desc == "NNLM":
        for batch in val_loader:
            logits = model.__call__(batch)
            log_probs = - logits.detach().cpu().numpy().tolist()
            surprisals.extend(log_probs[:])

    return surprisals



In [ ]:
memory_sizes = np.arange(3, 9)
print(f"Memory sizes to evaluate: {memory_sizes}")
groups = ['natural', 'complete_random', 'dep_preserved'] 
int2lang = {0:"en", 1:"spanish", 2:"des", 3:"turk", 4:"hungarian",
            5:"russia", 6:"jap", 7:"finnish", 8:"arabic", 9:"cn"}
langs = list(int2lang.values())
models = ['NNLM', 'RNNLM']
batch_size = 32
shuffle = True
average_surprisals = np.zeros((len(models), len(groups), len(langs), len(memory_sizes)))
std_surprisals = np.zeros((len(models), len(groups), len(langs), len(memory_sizes)))
num_epochs = 16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
all_surprisals = {model_name: {lang_name: {memory_size: {group: [] for group in groups} for memory_size in memory_sizes} for lang_name in langs} for model_name in models}
for model_idx, model_name in enumerate(models):
    print(f"Processing model: {model_name}")
    for lang_idx, lang_name in enumerate(langs):
        print(f"    Processing language: {lang_name}")
        for mem_idx, memory_size in enumerate(memory_sizes):
            print(f"      Memory size: {memory_size}")
            # get sentences
            corpora_text = corpora[lang_idx]
            # initialize tokenizer
            tokenizer = NaiveTokenizer(prepare_setence_for_tokenizer(corpora_text))
            # initialize dataset and dataloader
            # +1 memory size for target word
            naturalSentences, completeRandomSentences, depPreservedSentences = prepare_datasets(corpora_text, tokenizer, memory_size+1)
            trainNatural, testNatural = split_dataset(naturalSentences, split_ratio=0.8)
            trainCmpRan, testCmpRan = split_dataset(completeRandomSentences, split_ratio=0.8)
            trainDepPre, testDepPre = split_dataset(depPreservedSentences, split_ratio=0.8)
            trainDataLoaders = {
                'natural': DataLoader(trainNatural, batch_size=batch_size, shuffle=shuffle, collate_fn=tokenizer.pad_batch),
                'complete_random': DataLoader(trainCmpRan, batch_size=batch_size, shuffle=shuffle, collate_fn=tokenizer.pad_batch),
                'dep_preserved': DataLoader(trainDepPre, batch_size=batch_size, shuffle=shuffle, collate_fn=tokenizer.pad_batch),
            }
            test_loaders = {
                'natural': DataLoader(testNatural, batch_size=len(testNatural), shuffle=False, collate_fn=tokenizer.pad_batch),
                'complete_random': DataLoader(testCmpRan, batch_size=len(testCmpRan), shuffle=False, collate_fn=tokenizer.pad_batch),
                'dep_preserved': DataLoader(testDepPre, batch_size=len(testDepPre), shuffle=False, collate_fn=tokenizer.pad_batch),
            }

            # initialize model
            for group_idx, group in enumerate(groups):
                if model_name == 'RNNLM':
                    model = RNNLanguageModel(
                        vocab_size=tokenizer.vocab_size,
                        embedding_dim=32,
                        hidden_dim=64,
                        num_layers=2,
                        dropout_prob=0.5
                    )
                elif model_name == 'NNLM':
                    model = NNLM(
                        emb_size=32,
                        vocab_size=tokenizer.vocab_size,
                        hidden_size=64,
                        memory_size=memory_size,
                        pad_id=tokenizer.pad_id
                    )
                model.to(device)

                # training
                surprisals = train_and_evaluate(model, trainDataLoaders[group], test_loaders[group], num_epochs, learning_rate=0.001, device=device, type_desc=model_name)
                all_surprisals[model_name][lang_name][memory_size][group] = surprisals

                # store results
                average_surprisals[model_idx, group_idx, lang_idx, mem_idx] = np.mean(np.array(surprisals))
                std_surprisals[model_idx, group_idx, lang_idx, mem_idx] = np.std(np.array(surprisals)) 

path = "./data/all_surprisals.npz"
np.savez(path, all_surprisals=all_surprisals)
path = "./data/surprisal_results.npz"
np.savez(path,
         average_surprisals=average_surprisals,
         std_surprisals=std_surprisals,
         models=models,
         groups=groups,
         langs=langs,
         memory_sizes=memory_sizes)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
visualization_data = np.load("./data/surprisal_results.npz", allow_pickle=True)
average_surprisals = visualization_data['average_surprisals']
std_surprisals = visualization_data['std_surprisals']
models = visualization_data['models']
groups = visualization_data['groups']
langs = visualization_data['langs']
memory_sizes = visualization_data['memory_sizes']
# Surprisal vs Memory Size plots
# each row is a model, each column is a group
# each line in the plot is a language
# with error bars
num_models = len(models)
num_groups = len(groups)
fig, axes = plt.subplots(num_models, num_groups, figsize=(20, 10), sharex=True, sharey=True)
for model_idx in range(num_models):
    for group_idx in range(num_groups):
        ax = axes[model_idx, group_idx]
        for lang_idx, lang_name in enumerate(langs):
            avg_surprisals = average_surprisals[model_idx, group_idx, lang_idx, :]
            stds = std_surprisals[model_idx, group_idx, lang_idx, :]
            sns.lineplot(x=memory_sizes, y=avg_surprisals, ax=ax, label=lang_name)
        ax.set_title(f"Model: {models[model_idx]}, Group: {groups[group_idx]}")
        ax.set_xlabel("Memory Size")
        ax.set_ylabel("Average Surprisal")
        ax.legend()
        ax.grid()
plt.tight_layout()
plt.show()


In [ ]:
average_surprisals.shape
sns.heatmap(average_surprisals[0,1, :, :])

**Exercise 8 (1pt)** It would be tempting to use a multilingual large language model here. Is it possible ? conjecture whether they could be used or not ? what are the main challenges ?

multilingual-> different dependency locacity
working memory constraint: how many bits need to remember to process the next token
Memory–Surprisal Tradeoff curve（MST 曲线）: constraint memory capacity, the surprisal of the next token
information locacity: statistically depend-words are more close to each other \
dependency locacity: dependent elements are close to each other for memory efficiency